# Import all the necessary libraries

In [1]:
import torch
import numpy as np
from transformers import PatchTSTForPretraining
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
from torch.utils.data import Dataset, DataLoader

c:\Users\kmyut\anaconda3\envs\cs135_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data preprocessing
 - The earth FM expects 48 timesteps per pixel. Make sure to aggregate appropriately


In [2]:
import pandas as pd
# This is a dummy dataset
# N.B. This df is for illustration only and should only be used to get an understanding of the problem. This data is completely fictious.
train_data = pd.read_csv('Final_input.csv')
train_data = train_data.dropna()

counts = train_data['unique_id'].value_counts()

# Filter unique_ids that appear more than once
ids_to_keep = counts[counts > 1].index

# Keep only rows where unique_id appears more than once
train_data = train_data[train_data['unique_id'].isin(ids_to_keep)]
train_data.head()



,unique_id,time,x,y,crop_type,red,nir,swir16,swir22,blue,green,rededge1,rededge2,rededge3,nir08
0,Cameroon_agro-industrial_zones_1027296.3541211...,2023-01-28,1.027296e+06,442622.0456,cocoa,946.683944,1687.203380,1866.737465,1569.879437,530.815211,750.355493,1217.850704,1533.812958,1672.096901,1811.185352
1,Cameroon_agro-industrial_zones_1027296.3541211...,2025-02-26,1.027296e+06,442622.0456,cocoa,673.653371,2464.291011,1874.200562,1200.038202,439.839888,698.465169,1225.969101,2178.980337,2426.982584,2619.929775
2,Cameroon_agro-industrial_zones_1027296.3541211...,2021-01-28,1.027296e+06,442622.0456,cocoa,948.154366,2183.624789,2084.355493,1502.390423,605.953803,850.432113,1292.897465,1932.809014,2127.110423,2313.249577
3,Cameroon_agro-industrial_zones_1027296.3541211...,2022-01-03,1.027296e+06,442622.0456,cocoa,994.290141,1650.518310,2020.218028,1626.656338,569.026479,786.678310,1263.043380,1513.849577,1616.574085,1776.781972
4,Cameroon_agro-industrial_zones_1027296.3541211...,2025-02-21,1.027296e+06,442622.0456,cocoa,628.101408,2732.930704,1814.214648,1160.927324,488.134648,709.317183,1087.594930,2301.493521,2639.037746,2762.190986


In [ ]:
bands = {
    "red": "B4",
    "nir": "B8",
    "swir16": "B11",
    "swir22": "B12",
    "blue": "B2",
    "green": "B3",
    "rededge1": "B5",
    "rededge2": "B6",
    "rededge3": "B7",
    "nir08": "B8A",
}

reversed_bands = {v: k for k, v in bands.items()}


import glob
files = glob.glob('Input_csvs/*')
# print(files)


def fix_data(dataframes) :
    combined_df = pd.DataFrame(columns=train_data.columns)

    for df_name in dataframes:
        
        dataframe = pd.read_csv(df_name)

        if 'crop_type' not in dataframe.columns:
            dataframe['crop_type'] = 'rubber'
        try:
            dataframe.rename(columns=reversed_bands, inplace=True)
            dataframe = dataframe[train_data.columns]
            combined_df = pd.concat([combined_df, dataframe], ignore_index=True)

        except:
            print(df_name)
    return combined_df

combined = fix_data(files)
print(combined.head())
combined.to_csv("Final_input.csv", index = False)

In [3]:
test_data = pd.read_csv('test.csv')
test_data.head()

,unique_id,time,x,y,red,nir,swir16,swir22,blue,green,rededge1,rededge2,rededge3,nir08
0,ID_01FHV4,2018-01-03 10:59:22.851,-296455.0,846395.0,0.2920,0.3686,0.4173,0.3869,0.2488,0.2708,0.3211,0.3555,0.3752,0.3862
1,ID_01FHV4,2018-01-03 10:59:22.851,-296455.0,846395.0,0.2920,0.3686,0.4173,0.3869,0.2488,0.2708,0.3211,0.3555,0.3752,0.3862
2,ID_01FHV4,2018-02-12 10:59:25.232,-296455.0,846395.0,0.3510,0.3426,0.4817,0.4577,0.2538,0.2914,0.3684,0.3484,0.3588,0.3628
3,ID_01FHV4,2018-02-12 10:59:25.232,-296455.0,846395.0,0.3510,0.3426,0.4817,0.4577,0.2538,0.2914,0.3684,0.3484,0.3588,0.3628
4,ID_01FHV4,2018-03-14 10:59:24.436,-296455.0,846395.0,0.5312,0.6296,0.6643,0.5882,0.5244,0.5308,0.6016,0.6217,0.6401,0.6404


In [4]:
train_data['crop_type'].value_counts()

crop_type
rubber      40423
palm_oil    37921
cocoa       21100
Name: count, dtype: int64

In [5]:
timesteps_per_pixel = train_data.groupby('unique_id').size()  #count the timesteps per pixel since model expects 48 timesteps per pixel

print("Total timesteps per pixel:", len(timesteps_per_pixel))
print("Minimum timesteps per pixel:", timesteps_per_pixel.min())
print("Maximum timesteps per pixel:", timesteps_per_pixel.max())
print("Average timesteps per pixel:", timesteps_per_pixel.mean())

Total timesteps per pixel: 4456
Minimum timesteps per pixel: 2
Maximum timesteps per pixel: 96
Average timesteps per pixel: 22.316876122082586


In [6]:
test_pixel = test_data.groupby('unique_id').size()  #count the timesteps per pixel since model expects 48 timesteps per pixel

print("Total timesteps per pixel:", len(test_pixel))
print("Minimum timesteps per pixel:", test_pixel.min())
print("Maximum timesteps per pixel:", test_pixel.max())
print("Average timesteps per pixel:", test_pixel.mean())

Total timesteps per pixel: 10523
Minimum timesteps per pixel: 74
Maximum timesteps per pixel: 740
Average timesteps per pixel: 114.33155944122399


In [7]:
bands = ['red', 'nir', 'swir16', 'swir22', 'blue', 'green','rededge1', 'rededge2', 'rededge3', 'nir08']  # The spectral bands in the dataset

In [8]:
# You can have different ways to aggregate data to the required timesteps
# Below we have use interpolation, but remember it might introduce noise

from scipy.interpolate import CubicSpline
import numpy as np
import pandas as pd

def preprocess_with_interpolation(df, bands):
    all_results = []
    has_crop_type = 'crop_type' in df.columns

    for pixel_id, group in df.groupby('unique_id'):
        group = group.sort_values('time').reset_index(drop=True)

        # if len(group) < 2:
        #     # Not enough points to interpolate; either skip or handle differently
        #     print(f"Skipping unique_id {pixel_id} because it has less than 2 time points.")
        #     continue

        if len(group) == 48:
            keep_cols = ['unique_id', 'x', 'y'] + bands
            if has_crop_type:
                keep_cols.insert(1, 'crop_type')

            clean_group = group[keep_cols].copy()
            clean_group['timestep'] = range(48)
            all_results.append(clean_group)
        else:
            new_rows = []
            interpolated_bands = {}
            old_times = np.arange(len(group))
            new_times = np.linspace(0, len(group) - 1, 48)

            for band in bands:
                spline = CubicSpline(old_times, group[band].values)
                interpolated_bands[band] = spline(new_times)

            for i in range(48):
                new_row = {
                    'unique_id': pixel_id,
                    'timestep': i,
                    'x': group['x'].iloc[0],
                    'y': group['y'].iloc[0]
                }
                if has_crop_type:
                    new_row['crop_type'] = group['crop_type'].iloc[0]

                for band in bands:
                    new_row[band] = interpolated_bands[band][i]

                new_rows.append(new_row)

            all_results.append(pd.DataFrame(new_rows))

    final_df = pd.concat(all_results, ignore_index=True)
    return final_df


In [9]:
import numpy as np
cols_to_check = bands

# Check for NaN in those columns
nan_mask = train_data[cols_to_check].isna()

# Check for inf or -inf in those columns
inf_mask = ~np.isfinite(train_data[cols_to_check])

# Combine both masks
bad_mask = nan_mask | inf_mask

# Count bad values per selected column
print("NaN or Inf count per specified columns:")
print(bad_mask.sum())

# Show rows where any of those columns have bad values
bad_rows = train_data[bad_mask.any(axis=1)]
print("\nRows with NaN or Inf in specified columns:")
print(bad_rows)

NaN or Inf count per specified columns:
red         0
nir         0
swir16      0
swir22      0
blue        0
green       0
rededge1    0
rededge2    0
rededge3    0
nir08       0
dtype: int64

Rows with NaN or Inf in specified columns:
Empty DataFrame
Columns: [unique_id, time, x, y, crop_type, red, nir, swir16, swir22, blue, green, rededge1, rededge2, rededge3, nir08]
Index: []


In [10]:
preprocessed_train_data = preprocess_with_interpolation(train_data, bands)
preprocessed_test_data = preprocess_with_interpolation(test_data, bands)

In [11]:
train_pixel = preprocessed_train_data.groupby('unique_id').size()  #count the timesteps per pixel since model expects 48 timesteps per pixel

print("Minimum timesteps per pixel:", train_pixel.min())
print("Maximum timesteps per pixel:", train_pixel.max())


Minimum timesteps per pixel: 48
Maximum timesteps per pixel: 48


# MODELLING
- The earth FM was pretrained on a vast amount of Sentinel 2 unlabeled timeseries data, built on [PATCHTST](https://huggingface.co/docs/transformers/en/model_doc/patchtst#transformers.PatchTSTForPretraining) architecture.
- The pretrained model can be used in different ways: finetuning though supervised classification, as a feature extractor etc.

Download the models on hugging face
 - [600K](https://huggingface.co/AminiTech/FM-600K)
 - [18M](https://huggingface.co/AminiTech/fm-v2-28M)

The model expects a dataset and its mask as input

In [12]:
# use the model as a feature extractor for RF model
def extract_patch_embeddings(model, past_values):
    past_observed_mask = ~torch.isnan(past_values)

    with torch.no_grad():
        model_output = model.model(
            past_values=past_values,
            past_observed_mask=past_observed_mask,
            return_dict=True
        )
        embeddings = model_output.last_hidden_state
        all_patches = embeddings[:, :, 1:, :]
        final_embeddings = all_patches.mean(dim=(1, 2))  # average of all patch level embeddings
        return final_embeddings


In [13]:
hf_token = os.getenv('HUGGINGFACE_HUB_TOKEN')
MODEL_PATH = "AminiTech/fm-v2-28M"


In [14]:
class CropDataset(Dataset):
    def __init__(self, sequences, labels, unique_ids):
        self.sequences = torch.FloatTensor(sequences)
        self.labels = labels
        self.unique_ids = unique_ids

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        item = {
            'sequence': self.sequences[idx],
            'unique_id': self.unique_ids[idx]
        }
        if self.labels is not None:
            item['label'] = self.labels[idx]
        return item


def prepare_sequences_from_df(df):
    sequences = []
    labels = []
    unique_ids = []
    has_labels = 'crop_type' in df.columns

    for unique_id, group in df.groupby('unique_id'):
        spectral_data = group[bands].values
        sequences.append(spectral_data)
        unique_ids.append(unique_id)

        if has_labels:
            labels.append(group['crop_type'].iloc[0])

    sequences = np.array(sequences)

    print(f"Prepared {len(sequences)} sequences")

    if has_labels:
        print(f"Crop distribution:")
        unique, counts = np.unique(labels, return_counts=True)
        for crop, count in zip(unique, counts):
            print(f"  {crop}: {count}")
    else:
        labels = None

    return sequences, labels, unique_ids

def extract_embeddings_from_dataloader(model, dataloader):
    model.eval()
    all_embeddings = []
    all_ids = []

    device = next(model.parameters()).device

    with torch.no_grad():
        for batch in dataloader:
            sequences = batch['sequence'].to(device)
            embeddings = extract_patch_embeddings(model, sequences)

            all_embeddings.append(embeddings.cpu())  # Move to CPU before collecting
            all_ids.extend(batch['unique_id'])

    return torch.cat(all_embeddings, dim=0), all_ids

In [15]:
train_seq, train_labels, train_ids = prepare_sequences_from_df(preprocessed_train_data)
train_dataset = CropDataset(train_seq, train_labels, train_ids)


Prepared 4456 sequences
Crop distribution:
  cocoa: 601
  palm_oil: 2390
  rubber: 1465


In [16]:
print("Train sequences shape:", train_seq.shape)


Train sequences shape: (4456, 48, 10)


In [17]:
test_seq, test_labels, test_ids = prepare_sequences_from_df(preprocessed_test_data)
test_dataset = CropDataset(test_seq, test_labels, test_ids)

Prepared 10523 sequences


In [18]:
print("Test sequences shape:", test_seq.shape)

Test sequences shape: (10523, 48, 10)


In [19]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [20]:
# Load model
model = PatchTSTForPretraining.from_pretrained(MODEL_PATH, token=hf_token)
model.eval()

c:\Users\kmyut\anaconda3\envs\cs135_env\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\kmyut\anaconda3\envs\cs135_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kmyut\.cache\huggingface\hub\models--AminiTech--fm-v2-28M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as

PatchTSTForPretraining(
  (model): PatchTSTModel(
    (scaler): PatchTSTScaler(
      (scaler): PatchTSTMeanScaler()
    )
    (patchifier): PatchTSTPatchify()
    (masking): PatchTSTMasking()
    (encoder): PatchTSTEncoder(
      (embedder): PatchTSTEmbedding(
        (input_embedding): Linear(in_features=12, out_features=512, bias=True)
      )
      (positional_encoder): PatchTSTPositionalEncoding(
        (positional_dropout): Identity()
      )
      (layers): ModuleList(
        (0-5): 6 x PatchTSTEncoderLayer(
          (self_attn): PatchTSTAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout_path1): Identity()
          (norm_sublayer1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
 

In [21]:
train_embeddings, train_ids = extract_embeddings_from_dataloader(model, train_loader)
test_embeddings, test_ids = extract_embeddings_from_dataloader(model, test_loader)

In [22]:
print(len(train_ids))
print(train_ids)

4456
['gfw_oil_palm_v20191031_102.2893849804895_-0.6897149994975678', 'gfw_oil_palm_v20191031_110.88331487546637_-2.3610033807513844', 'gfw_oil_palm_v20191031_112.56090821787944_2.6988900644749663', 'rubber_310', 'gfw_oil_palm_v20191031_151.92767503185343_-3.206950000142342', 'gfw_oil_palm_v20191031_114.25228971076689_2.738717542230944', 'CIVKokao_-7.824862432111941_6.4108430665', 'rubber_739', 'gfw_oil_palm_v20191031_115.05860906781683_-1.2978521482579595', 'CIVKokao_-7.278198699485076_6.1376333059', 'gfw_oil_palm_v20191031_114.46128549449097_3.894284059661743', 'gfw_oil_palm_v20191031_103.47770999063749_-2.0740293624151036', 'gfw_oil_palm_v20191031_-10.935956429778036_6.838588542224215', 'gfw_oil_palm_v20191031_113.17921025510043_3.139666931433794', 'GHAKokao_-1.9394599488469388_5.599674893169014', 'gfw_oil_palm_v20191031_102.28587604635023_-0.42507899954608774', 'gfw_oil_palm_v20191031_120.78560092669571_-2.461086938811121', 'gfw_oil_palm_v20191031_100.67189658228821_1.6226539997323

In [23]:
print(f"Embeddings shape: Train {train_embeddings.shape}, Test {test_embeddings.shape}")

Embeddings shape: Train torch.Size([4456, 512]), Test torch.Size([10523, 512])


In [24]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
print(train_labels_encoded)

[0 0 0 ... 2 2 2]


In [25]:
rf_model = RandomForestClassifier(class_weight="balanced", random_state=42, n_estimators=100)
rf_model.fit(train_embeddings.numpy(), train_labels_encoded)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [ ]:
# Evaluate
test_predictions_encoded = rf_model.predict(test_embeddings.numpy())


In [ ]:
test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

submission = pd.DataFrame({
    'ID': test_ids,
    'Target': test_predictions
})
submission.head()
# submission.to_csv("submission.csv", index=False)

In [ ]:
import ee
ee.Initialize(project='gradient-growers')


In [ ]:
import geopandas

In [ ]:
longitude = 115.794902228198879
latitude = -1.629352395440254

point = ee.Geometry.Point(longitude, latitude)
start_date = '2024-01-01'
end_date = '2024-01-31'

# Example: Accessing Sentinel-2 Surface Reflectance data
collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(point) \
    .filterDate(start_date, end_date) \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first() # Get the least cloudy image

if collection:
    # Select specific bands (e.g., B4=Red, B3=Green, B2=Blue, B8=NIR for Sentinel-2)
    # Apply scaling factors if necessary (often provided in dataset metadata)
    # For Sentinel-2 SR, reflectance values are 10000 * actual reflectance
    bands_of_interest = ['B2', 'B3', 'B4', 'B8']
    image_at_point = collection.select(bands_of_interest).sample(point, scale=10).first()

    # Get the band values at the point
    data = image_at_point.getInfo()
    print(f"Band data at ({latitude}, {longitude}):")
    for band in bands_of_interest:
        if band in data['properties']:
            value = data['properties'][band]
            print(f"{band}: {value / 10000.0}") # Scale back to actual reflectance
else:
    print("No image found for the specified location and time range.")


In [ ]:
# collecting data from cocoa .tif
import rasterio
import matplotlib.pyplot as pyplot

CIV_cocao = 'Zindi\Cocoa\Detected_Cocoa_Farms\Detected Cocoa Farms\CIVKakao.tif'
GHA_cocao = 'Zindi\Cocoa\Detected_Cocoa_Farms\Detected Cocoa Farms\GHAKakao.tif'
try:
        with rasterio.open(CIV_cocao) as src:
                band_data = src.read(1)
                print(f'CIV - Coordinate Reference System (CRS): {src.crs}')
                pyplot.imshow(band_data, cmap='pink')
                pyplot.show()
        with rasterio.open(GHA_cocao) as src:
                band_data = src.read(1)
                print(f'GHA - Coordinate Reference System (CRS): {src.crs}')
except rasterio.errors.RasterioIOError as e:
        print(f'Error opening or reading GeoTIFF file: {e}')
except Exception as e:
        print(f'An unexpected error occurred: {e}')

In [ ]:
import rasterio
from rasterio.enums import Resampling
import pandas as pd
import numpy as np

CIV_cocao = 'Zindi/Cocoa/Detected_Cocoa_Farms/Detected Cocoa Farms/CIVKakao.tif'

try:
    with rasterio.open(CIV_cocao) as src:
        # Set a scale factor, e.g. 0.1 means 10% of the original resolution
        scale = 0.001
        out_shape = (
            1,
            int(src.height * scale),
            int(src.width * scale)
        )

        band_data = src.read(
            1,
            out_shape=out_shape,
            resampling=Resampling.nearest
        )

        transform = src.transform * src.transform.scale(
            (src.width / band_data.shape[1]),
            (src.height / band_data.shape[0])
        )

        print(f'CIV - Coordinate Reference System (CRS): {src.crs}')

        rows, cols = np.nonzero(band_data)
        lons, lats = rasterio.transform.xy(transform, rows, cols)
        df = pd.DataFrame({'Longitude': lons, 'Latitude': lats})
        print(df.head())
        df.to_csv('CIVKokao_coordinates_downsampled.csv', index=False)

except rasterio.errors.RasterioIOError as e:
    print(f'Error opening or reading GeoTIFF file: {e}')
except Exception as e:
    print(f'An unexpected error occurred: {e}')


In [ ]:
pip install pandas

In [ ]:
import rasterio
from rasterio.enums import Resampling
import pandas as pd
import numpy as np

GHA_cocao = 'Zindi/Cocoa/Detected_Cocoa_Farms/Detected Cocoa Farms/GHAKakao.tif'

try:
    with rasterio.open(GHA_cocao) as src:
        # Set a scale factor, e.g. 0.1 means 10% of the original resolution
        scale = 0.001
        out_shape = (
            1,
            int(src.height * scale),
            int(src.width * scale)
        )

        band_data = src.read(
            1,
            out_shape=out_shape,
            resampling=Resampling.nearest
        )

        transform = src.transform * src.transform.scale(
            (src.width / band_data.shape[1]),
            (src.height / band_data.shape[0])
        )

        print(f'GHA - Coordinate Reference System (CRS): {src.crs}')

        rows, cols = np.nonzero(band_data)
        lons, lats = rasterio.transform.xy(transform, rows, cols)
        df = pd.DataFrame({'Longitude': lons, 'Latitude': lats})
        print(df.head())
        df.to_csv('GHAKokao_coordinates_downsampled.csv', index=False)

except rasterio.errors.RasterioIOError as e:
    print(f'Error opening or reading GeoTIFF file: {e}')
except Exception as e:
    print(f'An unexpected error occurred: {e}')
